In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Generate synthetic dataset
np.random.seed(42)
n_samples = 1000
X = np.linspace(0, 10, n_samples).reshape(-1, 1)
y = np.sin(X) + 0.1*np.random.randn(n_samples, 1)

# Split dataset into training and calibration sets
X_train, X_calib, y_train, y_calib = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model using Random Forest regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Define conformal predictor function
def conformal_predict(model, X_train, y_train, X_test, eps):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    y_pred = model.predict(X_test)
    pred_interval = np.zeros((n_test, 2))
    for i in range(n_test):
        # Compute residuals for training set
        res_train = np.abs(y_train - model.predict(X_train))
        # Compute conformity scores for training set
        scores_train = np.sum(res_train <= res_train[i]) / n_train
        # Compute prediction interval for test point
        quantile = eps / (n_train + 1)
        lower = np.percentile(y_train, 100 * quantile / 2)
        upper = np.percentile(y_train, 100 * (1 - quantile / 2))
        width = upper - lower
        pred_interval[i, 0] = y_pred[i] - width/2
        pred_interval[i, 1] = y_pred[i] + width/2
    return pred_interval

# Compute prediction intervals for calibration set
y_calib_pred = conformal_predict(model, X_train, y_train, X_calib, eps=0.1)

# Compute calibration error rate
n_calib = y_calib.shape[0]
error_rate = np.mean((y_calib < y_calib_pred[:, 0]) | (y_calib > y_calib_pred[:, 1]))

print("Calibration error rate:", error_rate)

# Visualize prediction intervals for test set
X_test = np.linspace(0, 10, 100).reshape(-1, 1)
y_test = np.sin(X_test)
y_test_pred = conformal_predict(model, X_train, y_train, X_test, eps=0.1)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
plt.plot(X_test, y_test, 'r-', label='True function')
plt.plot(X_test, y_test_pred[:, 0], 'b--', label='Lower bound')
plt.plot(X_test, y_test_pred[:, 1], 'g--', label='Upper bound')
plt.fill_between(X_test.ravel(), y_test_pred[:, 0], y_test_pred[:, 1], alpha=0.2)
plt.legend()
plt.show()
